In [1]:
from collections import deque

def find_shortest_path(matrix):
    #               Up,      Down,  Left,   Right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Initialize positions
    start = (1, 1)  # Ali's position
    end = (4, 4)    # Home position

    # Checking if the start or the goal position is blocked
    if matrix[start[0]][start[1]] == 1 or matrix[end[0]][end[1]] == 1:
        return None

    # Initialize BFS data structures
    queue = deque()
    queue.append((start, [start]))  # Storing the current position and the path
    visited = set()
    visited.add(start)

    # BFS Loop
    while queue:
        (current, path) = queue.popleft()  # Get the next position and its path

        # Check if HOME is reached
        if current == end:
            return path  # Return the shortest path

        # Explore neighboring cells (non-diagonal and not obstacles)
        for direction in directions:
            next_row = current[0] + direction[0]
            next_col = current[1] + direction[1]
            next_pos = (next_row, next_col)

            # Check if the next position is valid and not visited
            if (0 <= next_row < len(matrix) and 0 <= next_col < len(matrix[0]) and
                matrix[next_row][next_col] == 0 and next_pos not in visited):
                visited.add(next_pos)  # Mark the position as visited
                queue.append((next_pos, path + [next_pos]))  # Add to queue with updated path

    # If no path is found
    return None


# Matrix showing the obstacles with digit 1
matrix = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0]
]

shortest_path = find_shortest_path(matrix)
if shortest_path:
    print("Shortest Path:", shortest_path)
else:
    print("No path found.")

Shortest Path: [(1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (4, 4)]


In [1]:
import time

def state_to_tuple(state):
    """Convert a string state to a tuple representation."""
    return tuple([tuple([state[i * 3 + j] for j in range(3)]) for i in range(3)])

def tuple_to_state(matrix):
    """Convert a tuple representation back to a string state."""
    return ''.join(''.join(row) for row in matrix)

def get_moves(matrix):
    """Generate possible moves from the given state."""
    moves = []
    blank_pos = None
    for i in range(3):
        for j in range(3):
            if matrix[i][j] == '0':
                blank_pos = (i, j)
                break
        if blank_pos:
            break

    i, j = blank_pos
    # Move Up
    if i > 0:
        new_matrix = [list(row) for row in matrix]
        new_matrix[i][j], new_matrix[i - 1][j] = new_matrix[i - 1][j], new_matrix[i][j]
        moves.append(tuple([tuple(row) for row in new_matrix]))
    # Move Down
    if i < 2:
        new_matrix = [list(row) for row in matrix]
        new_matrix[i][j], new_matrix[i + 1][j] = new_matrix[i + 1][j], new_matrix[i][j]
        moves.append(tuple([tuple(row) for row in new_matrix]))
    # Move Left
    if j > 0:
        new_matrix = [list(row) for row in matrix]
        new_matrix[i][j], new_matrix[i][j - 1] = new_matrix[i][j - 1], new_matrix[i][j]
        moves.append(tuple([tuple(row) for row in new_matrix]))
    # Move Right
    if j < 2:
        new_matrix = [list(row) for row in matrix]
        new_matrix[i][j], new_matrix[i][j + 1] = new_matrix[i][j + 1], new_matrix[i][j]
        moves.append(tuple([tuple(row) for row in new_matrix]))

    return moves

def dfs(start_state, goal_state):
    """Perform Depth-First Search (DFS) to find a solution path."""
    stack = [(start_state, [start_state])]
    visited = set()
    visited.add(start_state)

    while stack:
        current_state, path = stack.pop()
        if current_state == goal_state:
            return path
        for move in get_moves(current_state):
            if move not in visited:
                visited.add(move)
                stack.append((move, path + [move]))
    return None

def main():
    """Main function to take input and execute the DFS algorithm."""
    start_state = input("Enter start State: ")
    goal_state = input("Enter goal State: ")
    start_tuple = state_to_tuple(start_state)
    goal_tuple = state_to_tuple(goal_state)
    print("-----------------")
    print("DFS Algorithm")
    print("-----------------")
    start_time = time.time()
    solution_path = dfs(start_tuple, goal_tuple)
    end_time = time.time()
    if solution_path:
        print("Time taken:", end_time - start_time, "seconds")
        print("Path Cost:", len(solution_path) - 1)
        print("No of Nodes Visited:", len(solution_path))
        for state in solution_path:
            for row in state:
                print(' '.join(row))
            print("------")
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()

Enter start State: 120345678
Enter goal State: 012345678
-----------------
DFS Algorithm
-----------------
Time taken: 4.267692565917969e-05 seconds
Path Cost: 2
No of Nodes Visited: 3
1 2 0
3 4 5
6 7 8
------
1 0 2
3 4 5
6 7 8
------
0 1 2
3 4 5
6 7 8
------


In [4]:
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        """Initializes the graph with an adjacency list."""
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        """Returns the neighbors of a given node."""
        return self.adjacency_list.get(v, [])

    def compute_heuristic(self, stop_node):
        """Computes the heuristic value (distance to the goal node) for all nodes using BFS."""
        heuristic = {}
        queue = deque([stop_node])
        heuristic[stop_node] = 1  # Distance to itself is 1

        # Performing BFS to compute heuristic values
        while queue:
            current_node = queue.popleft()
            for neighbor, _ in self.get_neighbors(current_node):
                if neighbor not in heuristic:
                    heuristic[neighbor] = heuristic[current_node] + 1
                    queue.append(neighbor)

        # Assign a default heuristic value for nodes not reachable from the stop_node
        for node in self.adjacency_list:
            if node not in heuristic:
                heuristic[node] = float('inf')  # infinite heuristic for unreachable nodes

        return heuristic

    def a_star_algorithm(self, start_node, stop_node):
        """Implements the A* search algorithm to find the optimal path."""
        # Compute heuristic values for all nodes
        self.heuristic = self.compute_heuristic(stop_node)

        open_list = set([start_node])
        closed_list = set([])

        g = {}  # Cost from start node to all other nodes
        g[start_node] = 0

        parents = {}  # Keeping track of path
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None  # Current node

            # Find the node with the lowest using f(n) = g(n) + h(n)
            for v in open_list:
                if n == None or g[v] + self.heuristic[v] < g[n] + self.heuristic[n]:
                    n = v

            if n == None:
                print("Path does not exist!")
                return None

            # when the goal is found, reconstruct and return the path
            if n == stop_node:
                reconst_path = []
                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]
                reconst_path.append(start_node)
                reconst_path.reverse()
                print("Sentence:", ' '.join(reconst_path))
                print("Total cost:", g[stop_node])
                return reconst_path

            # Explore all neighbors of the current node
            for (m, weight) in self.get_neighbors(n):
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            open_list.remove(n)
            closed_list.add(n)

        print("Path does not exist!")
        return None

# Define the graph using the adjacency list
adjacency_list = {
    'The': [('cat', 2), ('dog', 3)],
    'cat': [('runs', 1)],
    'dog': [('runs', 2)],
    'runs': [('fast', 2)],
    'fast': []
}

graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('The', 'fast')  # Define start and goal words

Sentence: The cat runs fast
Total cost: 5


['The', 'cat', 'runs', 'fast']